# Data poisoning

## Roy Dong

In this lab, we're going to experiment with data poisoning. At a very high level, we suppose that our system designer wants to solve a machine learning problem of some sort. The system designer trains some algorithm to take input features and provide estimates of the output. 

(If the outputs are discrete, e.g. `(cat,dog)` or `(0,1,2,3,4,5,6,7,8,9)`, then this is a classification problem. If the outputs are continuous, then this is a regression problem.)

We suppose the adversary knows what algorithm the system designer uses. Furthermore, the adversary is able to manipulate a single data point to try and make the system perform poorly 'in the wild', i.e. on real world data.

## Data poisoning overview

Throughout this lab, we'll focus on data poisoning methods that follow a particular structure. It's very uncommon for data poisoning problems to be convex or other desirable optimization properties, so what we will do is simply perform gradient descent. In many settings, this is the 'best that we can do'.

To set things up, we suppose we're given some training data $(x_i,y_i)$ for $i = 1,\dots,n$. The $x_i$ are thought of as the input features and the $y_i$ are the labels or target values. Our adversary has the power to add one data point $(x_c,y_c)$ to the training data before the algorithm begins training. We'll let $X_{tr}$ and $Y_{tr}$ denote the training data.

We want to poison the algorithm to do badly in the wild. Our best approximation of the 'wild' is our validation dataset, which we will denote $(x_k,y_k)$ for $k = 1,\dots,m$. This validation dataset is **independent** of the algorithm training, and is used to evaluate the trained algorithm's performance on real world data. Similarly to before, we'll let $X_{val}$ and $Y_{val}$ denote the validation dataset.

So, formally, the adversary wants to maximize the system designer's loss on the validation data.

$$
L(X_{val},Y_{val};X_{tr},Y_{tr},x_c,y_c) =
\sum_{k = 1}^m \ell( f(x_k; X_{tr},Y_{tr},x_c,y_c), y_k )
$$

Here, we have some loss function $L$ which takes in datasets, and also a loss evaluated per data point, $\ell$. In the case of SVMs, a typical loss function is the hinge loss, which is equal to $\ell(f(x),y) = \max(0,1 - y f(x)) = (1 - y f(x))_+$. In the case of linear regression, the most common loss function, by far, is $\ell(f(x),y) = \| f(x) - y \|_2^2$.

Generally, we view the training and validation datasets as fixed, so we'll drop the dependencies. More succinctly, we have:
$$
L(x_c,y_c) = 
\sum_{k = 1}^m \ell( f(x_k; x_c,y_c), y_k )
$$

So, generally, this is the structure our data poisoning methods will follow.

    Initialize xc and yc to some initial point, xc0, yc0.
    Pick a step size t.
    
    While a termination condition is not satisfied...
    
        Calculate or approximate the gradient (gradLx,gradLy) of the loss w.r.t. (xc, yc).
    
        Update:
        xc = xc + t gradLx
        yc = yc + t gradLy
        
    Repeat.
    
In the case of data poisoning for SVMs, the termination condition is that the loss grows by less than some parameter $\epsilon$. In fact, the gradient sometimes does not even exist, but we can still use our approximation methods to use a surrogate for the gradient that works well enough in practice.

In a previous homework, you implemented finite difference approximations of the gradient, and this is how we will approximate the gradient throughout this lab.

Note that finite differences and gradient ascent are not the only ways to generate adversarial data points, but they are the only method we are covering in this lab.

# Data poisoning on SVMs using finite difference approximations

In this section, we'll instantiate our data poisoning framework on SVMs. In particular, we look at a binary classification problem. We suppose we can add one data point with the label $y_c = +1$, and are free to choose the $x_c$ that corresponds to this label $y_c = +1$.

This was discussed in class lecture, and is loosely based on the work in Biggio, Nelson, and Laskov's "Poisoning Attacks Against Support Vector Machines". The paper can be downloaded from here: https://arxiv.org/abs/1206.6389

First, we'll introduce some packages from `sklearn.svm` so that we can run our computations much more efficiently. Note that to do finite difference approximations of the gradient, we'll have to fit SVMs multiple times just to calculate the gradient once, so it will save us a lot of headache to do this efficiently.

## Initialize the problem

As always, you can run the cells below to initialize the problem.

In [ ]:
# load packages

import csv
import time

import numpy as np

import matplotlib.pyplot as plt

from sklearn import svm

In [ ]:
# helper functions

def plot_2D_classifier(beta,beta0,lt = 'k',lw = 4):
    
    x1=np.linspace(-20,20,1000)
    x2=-1*(beta[0]*x1+beta0)/beta[1]
    plt.plot(x1,x2,lt,linewidth=lw)
    return

In [ ]:
# generate data

np.random.seed(1337)

n_samples = 100
n_samples_validation = 100
n_features = 2

slope = 1
bias = 30

# randomly sample 0 or 1
y = np.random.binomial(n=1,p=0.5,size=n_samples)
# shift such that it's -1 and +1
y = np.subtract( np.multiply(y,2), 1 )

# randomly generate X in 2D
X = np.empty( (n_samples,2) )
X[:,0] = np.random.uniform(-20,20,n_samples)
X[:,1] = slope * X[:,0] + np.random.normal(0,10,n_samples)
X[:,1][y > 0] += bias

# randomly generate validation data as well
y2 = np.random.binomial(n=1,p=0.5,size=n_samples_validation)
y2 = np.subtract( np.multiply(y2,2), 1 )

X2 = np.empty( (n_samples_validation,2) )
X2[:,0] = np.random.uniform(-20,20,n_samples_validation)
X2[:,1] = slope * X2[:,0] + np.random.normal(0,10,n_samples_validation)
X2[:,1][y2 > 0] += bias

## Using sklearn.svm

In the previous assignments, we implemented SVMs using `scipy.optimize`. There's a lot of structure in SVMs that were not exploited in this implementation. Since we'll have to repeatedly fit SVM models to design a data poisoning attack, we will now use `sklearn.svm` to fit SVMs, as it finishes its computations much more quickly.

Recall that the soft-margin SVM is the solution to the optimization:
$$
\min_{\beta, \beta_0}~\frac{1}{2} \| \beta \|_2^2 + C \sum_{i = 1}^n \xi_i
$$
$$
\text{subject to}~y_i (x_i^\top \beta + \beta_0) \geq 1 - \xi_i \text{ for } i = 1,\dots,n
$$
$$\xi_i \geq 0 \text{ for } i = 1,\dots,n
$$

`sklearn.svm.SVC` takes in a `C` parameter, which corresponds to the penalty weight $C$ in the above optimization.

Also, we note that `sklearn.svm.SVC` is designed to handle multiclass SVMs, which can classify data into more than two categories. We will only be using the two-class functionality. There are many methods in `sklearn.svm.SVC` which return `n_classes-1` variables. For example, `clf.dual_coef_` gives the dual variables of `n_classes-1` different classes. In our case, this only returns one set of dual variables, as we covered in class. Thus, you may see throughout this code lines such as `clf.dual_coef_[0]` or `clf.coef_[0]`, which corresponds to us taking the first element of a `(1,n_features)` array and making it into an `(n_features,)` array.

Finally, it's also important to note that the dual variables returned by `sklearn.svm` are not the same as the ones covered in class, so it will be helpful to look into it a little bit.

In [ ]:
# using sklearn's svm functionality, let's fit a model

C = 2.0
clf = svm.SVC(C = C, kernel = 'linear')
clf.fit(X,y)

In [ ]:
# after fitting, let's extract out the model parameters

# clf.coef_ gives the beta coefficients for multiple classes
#   in the 2-class case, we get a 1 by n_features array
#   we want an n_features array, so let's just take the first element only
beta = clf.coef_[0]
beta0 = clf.intercept_

print('beta = \n {}'.format(beta))
print('beta0 = \n {}'.format(beta0))

## Support vectors

Recall in our discussion that for an SVM, all the data fits into three categories: margin support vectors, error support vectors, and reserve vectors.

For margin support vectors, we have $y_i (x_i^\top \beta + \beta_0) = 1$.

For error support vectors, we have $y_i (x_i^\top \beta + \beta_0) < 1$.

For reserve vectors, we have $y_i (x_i^\top \beta + \beta_0) > 1$.

In the Biggio et al paper, the authors refer to these sets of vectors as $\mathcal{S}$ for margin support vectors, $\mathcal{E}$ for error support vectors, and $\mathcal{R}$ for reserve vectors.

In [ ]:
# after fitting, let's extract out support vector information

# the indices of X which are support vectors
support_indices = clf.support_

# the values of the support vectors
support_vectors = clf.support_vectors_

# the values of the dual variables
dual_vars = clf.dual_coef_[0]

# let's print out some of the results from fitting, and interpret them
print('the indices of the support vectors are: \n{}'.format(support_indices))
print('the labels for these support vectors are: \n{}'.format(y[support_indices]))
print('the support vectors themselves are: \n{}'.format(support_vectors))
print()

print('the SVM decision function gives the following values at the support vectors:')
print('{}'.format(clf.decision_function(support_vectors)))
print('again, the labels for these support vectors are: \n{}'.format(y[support_indices]))
print()

print('recall that if the decision function is < 0, then it gets a label -1')
print('and if the decision function is > 0, then it gets a label +1')
print()

print('the dual variables (as defined by libsvm) are: \n{}'.format(dual_vars))
print('note that these dual variables are not exactly the same as the ones covered in class.')
print()

print('looking at the SVM decision function values and the dual variables,')
print('how can we determine the margin support vectors from the dual variable values?')

# Task: find the margin support vectors

Implement the code below to extract the margin support vectors from the array of support vectors.

In [ ]:
def find_margin_support_vectors(support_vectors,support_indices,labels,dual_vars,C):
    
#     TODO: write a function that returns the support vectors and their labels

    return margin_support_vectors, margin_indices, margin_labels

margin_support_vectors, margin_indices, margin_labels = find_margin_support_vectors(support_vectors,support_indices,y[support_indices],dual_vars,C)

print('the margin support vectors are: \n{}'.format(margin_support_vectors))
print('the margin support vectors are indexed at: \n{}'.format(margin_indices))
print('their labels are: \n{}'.format(margin_labels))
print('do these results match what you expected, looking at the data above?')

In [ ]:
# plot results from SVM fitting

Xp = X[y > 0]
Xn = X[y < 0]

plt.plot(Xp[:,0],Xp[:,1],'ro')
plt.plot(Xn[:,0],Xn[:,1],'bo')
plt.title('raw data')

# get the y limits and set all the graphs to the same ylim
yl, yr = plt.ylim()

plt.show()

plot_2D_classifier(beta,beta0)

plt.plot(Xp[:,0],Xp[:,1],'ro')
plt.plot(Xn[:,0],Xn[:,1],'bo')
plt.plot(support_vectors[:,0],support_vectors[:,1],'y*',markersize=10)
plt.title('data with classifier and support vectors')

# set ylim
plt.ylim(yl, yr)

plt.show()

plot_2D_classifier(beta,beta0)

plt.plot(support_vectors[:,0],support_vectors[:,1],'y*',markersize=10)
plt.title('classifier and support vectors')

# set ylim
plt.ylim(yl, yr)

plt.show()

plot_2D_classifier(beta,beta0)

plt.plot(margin_support_vectors[:,0],margin_support_vectors[:,1],'y*',markersize=10)
plt.title('classifier and margin support vectors')

# set ylim
plt.ylim(yl, yr)

plt.show()

# Calculating the hinge loss

Now, recall we had a loss function defined as:
$$
L(x_c) = \sum_{k = 1}^m (1 - y_k f(x_k;x_c))_+
$$
(Here, we've even dropped the dependence on $y_c$ since we view this as fixed here.) 

This loss is calculated across the validation dataset $(x_k,y_k)_{k = 1}^m$. We've generated validation data in the variables `X2` and `y2`, so let's calculate this hinge-loss.

Note that `clf.decision_function()` is the implementation of the function $f$ above.

# Task: calculate the hinge loss

Now, let's write a function that returns the value of $(1 - y_k f(x_k;x_c))_+$ for each $k$ (as variable `loss_per_data`) as well as $L(x_c)$.

In [ ]:
# calculate hinge loss

def calculate_hinge_loss(clf,X2,y2):
    
    # TODO: return loss and loss_per_data
    
    return loss, loss_per_data

loss,loss_per_data = calculate_hinge_loss(clf,X2,y2)

print('hinge loss per data point is: \n{}'.format(loss_per_data))
print('the hinge loss is: {}'.format(loss))

# Task: implement the loss function

Great, now we have all the parts, let's put it together to create a loss function.

Here, our loss function will simply be a mapping from training data, poisoned data point, and validation data to a scalar value (the total hinge loss).

In [ ]:
# implement loss function

def loss_function_SVM(Xtrain,ytrain,xc,yc,Xvalid,yvalid,C = 2.0):
    
    # TODO: given training data, poisoned data point, and validation data
    #   return the hinge loss on the validation data when the classifier
    #   is trained on the poisoned dataset
    
    return loss

# initialize xc and yc
xc = np.zeros(n_features)
yc = 1

loss_function_SVM(X,y,xc,yc,X2,y2)

# Task: implement finite differences function

Given a function $f$, we wish to use the finite differences approximation for the gradient, given by:
$$
\frac{\partial}{\partial x_i} f(x) \approx \frac{f(x + \frac{1}{2}h e_i) - f(x - \frac{1}{2}h e_i)}{h}
$$

(Note this can be directly copied from a previous homework.)

In [ ]:
# finite differences function

def finite_differences(f, x, h):
    
    return

# Stepping back

Let's take a pause here and re-orient ourselves. What have we done so far?

We've implemented a function that essentially takes in a poisoned data point and calculates the loss. We also have a finite difference approximation of the gradient of a function, given a function.

Recall our overall framework:

    Initialize xc and yc to some initial point, xc0, yc0.
    Pick a step size t.
    
    While a termination condition is not satisfied...
    
        Calculate or approximate the gradient (gradLx,gradLy) of the loss w.r.t. (xc, yc).
    
        Update:
        xc = xc + t gradLx
        yc = yc + t gradLy
        
    Repeat.
    
We have implemented code for `Calculate or approximate the gradient (gradLx,gradLy) of the loss w.r.t. (xc, yc).` Now, we simply have to perform gradient ascent.

# Gradient ascent details

## Using line search instead of a constant step size

Recall here that our termination condition will simply be when $L(x_c^+) - L(x_c) < \epsilon$.

Previously, we talked about a very basic version of gradient ascent/descent. We basically repeatedly iterate: $x^+ = x + t \nabla f(x)$ until our loss is sufficiently small.

However, we'll find that this doesn't work well on our problem. In fact, all the non-convexities and discontinuities are problematic for our purposes.

Instead, let us try the following. 
Let our direction be $d = \frac{\nabla f(x)}{\|\nabla f(x)\|_2}$ be the unit-norm vector pointing in the direction of the gradient. Let's consider the line connecting the points $x$ and $x + t d$. A line-search method is one where we look at points along this line and find the one that yields the highest value for what we are trying to maximize.

In other words, if we are given $x$, $t$, and $d$, we can pick out $t_{div}$ points from this line segment by considering the points:
$$
x
$$
$$
x + \frac{t}{t_{div}} d
$$
$$
x + 2 \frac{t}{t_{div}} d
$$
$$
\dots
$$
$$
x + (t_{div}-1)\frac{t}{t_{div}} d
$$
$$
x + t d
$$

We look at the losses at $L(x), L\left( x + \frac{t}{t_{div}} d \right), \dots$ and, since we are trying to maximize loss as an adversary, just pick the point among these that gives us the highest loss.

## Using random noise when the gradient is almost zero

Another finicky aspect of non-convex problems is that sometimes you might get stuck. Namely, your gradient is very small. This means that you are likely in a local optima, but this optima may not be very good. One thing you can do in these circumstances is to simply pick a random direction.

In your code, check if $\|\nabla f(x)\|_2$ is too small. In this case, replace it with a randomly generated unit-norm vector.

For your convenience, here is the code to generate a random unit-norm vector:

    direction = np.random.randn(n)
    direction = direction / np.linalg.norm(direction)

# Task: implement gradient ascent

Implement gradient descent with the two modifications described above.

I've added an argument called `verbose`. For debugging purposes, you can include statements of the form:

    if verbose:
      print('the value of ... is {}'.format(a))

This is a quick way to check on how your optimization is doing. When you're satisfied your implementation is good, just call the function with `verbose = False`.

Additionally, I've included a `MAX_ITER` argument. Make your code stop looping after `MAX_ITER` number of iterations, regardless of whether or not the termination condition has been met.

In [ ]:
# implement gradient ascent
# eps is the parameter for the termination condition
# h is the distance for the finite differences step
def gradient_ascent_via_finite_differences( f, t, x0, eps = 1e-3, h = 1., t_div = 25., MAX_ITER = 100, verbose = True ):
    
    if verbose:
        tic = time.time()
        
    # TODO
    
    if verbose:
        toc = time.time()
        
        print('total running time is {} seconds.\n'.format(toc - tic))
    
    return xopt

In [ ]:
# initialize xc and yc
xc = np.zeros(n_features)
yc = 1

L = lambda xc : loss_function_SVM(X,y,xc,yc,X2,y2,C = 2.0)
t = 5.
xopt = gradient_ascent_via_finite_differences(L,t,xc)
print('gradient ascent returns point \n{}\nwith loss {}'.format(xopt,L(xopt)))

# Task: fine tuning

Now, here's the essential part of any non-convex optimization problem: fine-tuning the parameters. Can you find worse adversarial examples using different initializations of $x_c$, $\epsilon$, $h$, `t_div`, and so on?

In [ ]:
# Find parameters that get your loss high.

# initialize xc and yc
# TODO: 
# xc = ???

# t = ???
# eps = ???
# t_div = ???

# xopt = gradient_ascent_via_finite_differences(L,t,xc,eps = eps, h = h, t_div = t_div)
# print('gradient ascent returns point \n{}\nwith loss {}'.format(xopt,L(xopt)))

# Task: plot results

Now that you've found a poisoned data point, can you visualize where the poisoned data point lives?

In [ ]:
# plot results from data poisoning


# From finite differences to using optimality conditions

Hopefully, up to this point in the lab, you have developed an understanding of how an adversary could use the gradient of the loss to increase the error of a classification algorithm. We did not explicitly calculate the gradient, but instead used finite difference approximations.

Since our toy example above was small, this was feasible to calculate. However, note that to do finite differences, we had to fit an SVM 2 times for each coordinate! This won't scale well with larger datasets. For example, consider the MNIST (Modified National Institute of Standards and Technology) database of handwritten digits.

In our toy example, we had to fit 4 SVMs to approximate the gradient once; if we were to try and use the MNIST dataset, there are $28^2 = 784$ features, so we would have to fit $1568$ SVMs to approximate the gradient once, and each of these SVMs will take longer to fit as well since they exist in a higher dimension.

This is what motivates the work in Biggio, Nelson, and Laskov's "Poisoning Attacks Against Support Vector Machines". The paper can be downloaded from here: https://arxiv.org/abs/1206.6389

Since this lab will be much more explicit in the derivation and development of the results, **at times our notation will differ from that of the source paper**. However, the best effort has been made to keep these as consistent as possible.

## Stepping back, one more time

Recall again, our overall framework:

    Initialize xc and yc to some initial point, xc0, yc0.
    Pick a step size t.
    
    While a termination condition is not satisfied...
    
        Calculate or approximate the gradient (gradLx,gradLy) of the loss w.r.t. (xc, yc).
    
        Update:
        xc = xc + t gradLx
        yc = yc + t gradLy
        
    Repeat.

Our claim just now was that this step `Calculate or approximate the gradient (gradLx,gradLy) of the loss w.r.t. (xc, yc).` was computationally expensive when the dimension is large. In this section, we make a few assumptions so that this can be more easily calculated.

## Looking at margin support vectors

We note again, all the data fits into three categories: margin support vectors, error support vectors, and reserve vectors.

For margin support vectors, we have $y_i (x_i^\top \beta + \beta_0) = 1$.

For error support vectors, we have $y_i (x_i^\top \beta + \beta_0) < 1$.

For reserve vectors, we have $y_i (x_i^\top \beta + \beta_0) > 1$.

In the Biggio et al paper, the authors refer to these sets of vectors as $\mathcal{S}$ for margin support vectors, $\mathcal{E}$ for error support vectors, and $\mathcal{R}$ for reserve vectors.

Recall that the authors assume that these sets remain constant for small perturbations in $x_c$, the poisoned data point. Let $x_i \in \mathcal{S}$ be a margin support vector and $y_i$ be its corresponding label, and let $x_c$ and $x_c'$ be two very close possible values for the poisoned data point. The authors claim:
$$
y_i (x_i^\top \beta(x_c) + \beta_0(x_c)) = 1
$$
$$
y_i (x_i^\top \beta(x_c') + \beta_0(x_c')) = 1
$$

Recall that our decision function is often denoted $f(x;x_c) = x^\top \beta(x_c) + \beta_0(x_c)$, so we can write this as:
$$
y_i f(x_i;x_c) = 1
$$
$$
y_i f(x_i;x_c') = 1
$$

Furthermore, since the authors assume that the set $\mathcal{S}$ is constant, it follows that this holds for **all** $x_c'$ that are close to $x_c$. So we can say that $y_i (x_i^\top \beta(x_c) + \beta_0(x_c))$ as a function of $x_c$ is **identically** equal to 1. This means its derivative is 0.

Let $(x_i,y_i)_{i = 1}^n$ denote our training set, and $(x_c,y_c)$ denote our poisoned data that we can insert.

If we plug in the optimality conditions, we can see that the decision function of our SVM is:
$$
f(x;x_c) = \left( \sum_{i = 1}^n \alpha_i y_i x_i + \alpha_c y_c x_c \right)^\top x + b
$$
(Here, we switch from $\beta_0$ to $b$, as per the Biggio et al notation.)

Now, consider the equality:
$$
y_i f(x_i;x_c) = \sum_{j = 1}^n Q_{ij} \alpha_j + Q_{ic} \alpha_c + y_i b
$$

(Be careful with the indices here; note that the sum is across $j$.)

If we are pattern matching, what are the values of $Q_{ij}$?

# Task: find the expression for $Q_{ij}$ and $Q_{ic}$

$$
Q_{ij} = ???
$$

$$
Q_{ic} = ???
$$

## Taking the derivatives

Now, we claimed that $y_i f(x_i;x_c) \equiv 1$ whenever $x_i \in \mathcal{S}$ is a margin support vector. This means its derivative with respect to $x_c$ is 0, since it is a function that does not change when $x_c$ changes. Essentially, the optimality condition holds at the old optimum with $x_c$ and the new optimum with $x_c'$.

Now, let's look at the derivative with respect to $x_c$ of $y_i f(x_i;x_c)$. (Again, looking only at margin support vectors.) Which terms depend on $x_c$?

Again, let $x_c = (u_1,\dots,u_d)$ be written in terms of its elements.

# Task: write this derivative out (scalars)

Note: write this first in terms of $\frac{\partial Q_{ic}}{\partial u_l}$.

$$
\frac{\partial}{\partial u_l} y_i f(x_i;x_c) = ??? = 0
$$

Again, for emphasis, we know that this equality holds **only for the margin support vectors**.

Furthermore, **we note that $\frac{\partial \alpha_j}{\partial u_l} = 0$ for any $j$ such that $x_j$ is either an error support vector or a reserve vector. Thus, we should think of the sum term as the sum across only terms from the margin support vectors**.

What is $\frac{\partial Q_{ic}}{\partial u_l}$? (Here, let $x_i = (v_1,\dots,v_d)$.)

$$
\frac{\partial Q_{ic}}{\partial u_l} = ???
$$


## Another optimality condition

Recall the other optimality condition we have:
$$
\sum_{i = 1}^n \alpha_i y_i + \alpha_c y_c = 0
$$

Since the old SVM (with $x_c$) and the new SVM (with $x_c'$) both are optimizers, we again have this optimality condition holds for all $x_c$. This means that this expression is **identically 0** regardless of $x_c$, so we also know the derivative of this constant function is 0. 

# Task: differentiate the above expression to get another equality constraint on the derivatives

$$
??? = 0
$$

# Task: make these equations into vectors, and then matrices

Let's fix $u_l$ to start. 

Suppose our margin support vectors, **not including our poisoned data point**, have indices $I_s$. (For example, if our margin support vectors were data points $1$, $5$, and $11$, then $I_s = \{1,5,11\}$.) In this example, let's define the vector:
$$
\frac{\partial \alpha}{\partial u_l} =
\begin{bmatrix}
\frac{\partial \alpha_s}{\partial u_l} \\
\frac{\partial \alpha_c}{\partial u_l} \\
\end{bmatrix} =
\begin{bmatrix}
\frac{\partial \alpha_1}{\partial u_l} \\
\frac{\partial \alpha_5}{\partial u_l} \\
\frac{\partial \alpha_{11}}{\partial u_l} \\
\frac{\partial \alpha_c}{\partial u_l} \\
\end{bmatrix}
$$
In other words, $\frac{\partial \alpha}{\partial u_l}$ is a vector of the non-zero derivatives of $\alpha$ and the derivative of $\alpha_c$. (Recall that for error support vectors and reserve vectors, the derivatives are 0.)

For $x_i \in \mathcal{S}$ that is a margin support vector, write the constraint $\frac{\partial}{\partial u_l} y_i f(x_i;x_c) = 0$ in vector notation.

That is, if we have:
$$
q_{is}^\top \frac{\partial \alpha}{\partial u_l} + \gamma_i \alpha_c + \zeta_i \frac{\partial b}{\partial u_l} = 0
$$

What are:

$$
q_{is}^\top =
\begin{bmatrix}
??? & ??? & \dots & ???
\end{bmatrix}
$$

$$
\gamma_i = ???
$$

$$
\zeta_i = ???
$$

**Make sure your dimensions match! That is, $\frac{\partial \alpha}{\partial u_l}$ has dimension equal to the number of margin support vectors plus one.**

Next, let:
$$
y =
\begin{bmatrix}
y_s \\
y_c
\end{bmatrix} =
\begin{bmatrix}
y_1 \\
y_5 \\
y_{11} \\
y_c
\end{bmatrix}
$$
When we consider the optimality condition $\sum_{i = 1}^n \alpha_i y_i + \alpha_c y_c = 0$, can you write the constraint that the derivative is 0 in vector form?
That is, if we have:
$$
q_i^\top \frac{\partial \alpha}{\partial u_l} = 0 
$$
What is $q_i^\top$?

$$
q_i^\top = ???
$$

Hopefully, you see how this generalizes for general $I_s$, not just when $I_s = \{1,5,11\}$.

# Task: make these equations into vectors, and then matrices (continued)

Now, can we put this all together? Namely, if we have:
$$
A \begin{bmatrix}
\frac{\partial \beta}{\partial u_l} \\
\frac{\partial \alpha}{\partial u_l}
\end{bmatrix} =
b
$$

What are $A$ and $b$?

**Hint: let $Q_{ss}$ denote a matrix whose $i,j$th entry corresponds to the $y_i y_j x_i^\top x_j$ of the $i$th and $j$th margin support vectors. Also, we can define $y_s$ as the vector containing only the margin support vector labels. Similarly, define $\frac{\partial Q_{sc}}{\partial u_l}$ as a vector whose dimension is the number of margin support vectors, such that the $i$th element has $\frac{\partial Q_{ic}}{\partial u_l}$, the partial derivative of the $i$th support vector.**

$$
A = 
???
$$

$$
b = 
???
$$

## Initialize poisoned data point and fit SVM to the poisoned dataset

Here's code to initialize the relevant variables.

In [ ]:
# initialize xc and yc
xc = np.zeros(n_features)
yc = 1

# make poisoned dataset
Xpoison = np.empty( (n_samples+1,n_features) )
Xpoison[0:-1] = X
Xpoison[-1] = xopt
    
ypoison = np.empty( n_samples+1 )
ypoison[0:-1] = y
ypoison[-1] = yc

print('data is now poisoned.')
print('poisoned data is at index -1.')

# Task: fit an SVM to the poisoned dataset and calculate the margin vectors

Write code to fit an SVM to this poisoned dataset, and the identify the margin support vectors.

To summarize what this code needs to do:

    fit an SVM to Xpoison, ypoison
    
    find the support indices, support vectors, and dual variables for this SVM
    
    calculate alpha_c
    
    find the margin support vectors, margin support indices, and margin labels
      store these in variables: margin_support_vectors, margin_indices, margin_labels
      
    additionally, determine if x_c is a margin support vector
      set poisoned_is_margin = True or = False accordingly
    
Note that in order to calculate $\alpha_c$, we must look at what type of data point $x_c$ is. If it is a reserve vector, what is $\alpha_c$? Similarly an error support vector? If it is a margin support vector, where can we find the value of $\alpha_c$?

In [ ]:
# fit an SVM to poisoned dataset and calculate margin vectors


# Task: implement these matrices

As hinted at in the previous task, we can consider a square matrix $Q_{ss}$ whose entries correspond to margin support vectors. Implement code to calculate these matrices.

Note that we have to handle the special case where $x_c$ is a margin or an error support vector! (If it is a margin support vector, the derivative will automatically be calculated in $Q_{ss}$; otherwise, we have to explicitly set it to zero.)

In [ ]:
def calculate_Qss(margin_support_vectors,margin_labels):
    
    # TODO
    
    return Q

Qss = calculate_Qss(margin_support_vectors,margin_labels)

print('The matrix Qss = \n{}'.format(Qss))

In [ ]:
def calculate_A(margin_support_vectors,margin_labels):
    
    return A

A = calculate_A(margin_support_vectors,margin_labels)

print('The matrix A = \n{}'.format(A))

In [ ]:
def calculate_b(alpha_c,yc,margin_support_vectors,margin_labels,l):
    
    # TODO
    
    return b

b = calculate_b(alpha_c,yc,margin_support_vectors,margin_labels,0)

print('The vector b = \n{}'.format(b))

In [ ]:
def calculate_coef_gradient(margin_support_vectors,margin_labels,alpha_c,yc,poisoned_is_margin = False):
    
    # TODO
    
    return partial_b, partial_alpha, partial_alpha_c

partial_b, partial_alpha, partial_alpha_c = calculate_coef_gradient(margin_support_vectors,margin_labels,alpha_c,yc,poisoned_is_margin)

print('partial_b = \n{}'.format(partial_b))
print('partial_alpha = \n{}'.format(partial_alpha))
print('  the i,j-th entry of partial_alpha gives partial alpha_i w.r.t. u_j')
print('  thus, the ith row of partial_alpha is the derivative of alpha_i w.r.t. x_c')
print('partial_alpha_c = \n{}'.format(partial_alpha_c))

## Hinge loss on validation data

Okay, we've come a very long way, and we're almost home free.

Now, recall we had a loss function defined as:
$$
L(x_c) = \sum_{k = 1}^m (1 - y_k f(x_k;x_c))_+
$$
This loss is calculated across the validation dataset $(x_k,y_k)_{k = 1}^m$. We've generated validation data in the variables `X2` and `y2`, so let's calculate this hinge-loss.

## Calculate the derivative of decision function with respect to $x_c$

So, we have:
$$
f(x;x_c) = \left( \sum_{i = 1}^n \alpha_i y_i x_i + \alpha_c y_c x_c \right)^\top x + b
$$
Recall that this summation across $x_i$ is across the **training data**, whereas the $x$ and $x_k$ arguments will typically be data from our **validation data**. 

What is the derivative of this with respect to $x_c$? Well, the terms that depend on $x_c$ are $\alpha$ and $b$. So:
$$
\frac{d}{d u_l} f(x_k;x_c) = \sum_{i = 1}^n \frac{\partial \alpha_i}{\partial u_l} y_i x_i^\top x_k + \frac{\partial \alpha_c}{\partial u_l} y_c x_c^\top x_k + \alpha_c y_c v_l + \frac{\partial b}{\partial u_l}
$$
Here, again $u_l$ is the $l$th component of $x_c$ and $v_l$ is the $l$th component of $x_k$.

Also, note that this summation across $i$ only needs to be across the margin support vectors, since the partial derivatives of the other terms are all 0.

# Task: calculate the derivative of the loss function with respect to $x_c$

There are a few cells that need coding now.

First, you must implement a function `decision_function_gradient` to implement the derivative of the decision function $f$. Then you must use this to calculate the variable `loss_gradient`, which is our final answer.

In [ ]:
# implement the derivative of decision function

def decision_function_gradient(partial_alpha,partial_b,partial_alpha_c,alpha_c,margin_support_vectors,margin_labels,xc,yc,xk):
    
    # TODO
    
    return f_grad

test = decision_function_gradient(partial_alpha,partial_b,partial_alpha_c,alpha_c,margin_support_vectors,margin_labels,xc,yc,X2[0])
print('the gradient of the decision function at X2[0] is: \n{}'.format(test))

In [ ]:
# calculate the gradient of the loss w.r.t. x_c now, finally!

loss_gradient = np.zeros(n_features)

# TODO

print('the gradient of the loss is: \n{}'.format(loss_gradient))

## Comment on the results

You'll probably find that your gradient is 0. In fact, if you think about it, the gradient will be zero in most locations. As such, it's a good thing our gradient ascent is coded such that it will explore a bit when the gradient actually equals 0.

# Putting it all together

I've intermittently repeated the overall framework so you can keep in mind the larger project at hand. We are doing the following:

    Initialize xc and yc to some initial point, xc0, yc0.
    Pick a step size t.
    
    While a termination condition is not satisfied...
    
        Calculate or approximate the gradient (gradLx,gradLy) of the loss w.r.t. (xc, yc).
    
        Update:
        xc = xc + t gradLx
        yc = yc + t gradLy
        
    Repeat.
    
We've finished calculating what the loss gradient is. Now let's wrap it up in a function and make a gradient ascent algorithm out of it!

# Task: implement the loss gradient approximation

This takes in `X`, `y`, `xc`, `yc`, `X2`, and `y2` to give you the gradient of the loss at that particular value of `xc`. This is taking the code above and making it a function that can be used by our gradient ascent in a little bit.

In [ ]:
def loss_gradient_approximation(X,y,xc,yc,X2,y2, C = 2.0):
    
    # TODO 
    
    return loss_gradient

# Task: implement gradient ascent which uses a gradient approximation

Now, replace the finite differences method with the gradient approximation method, which is passed in as the argument `fdot`.

In [ ]:
# implement gradient ascent
# eps is the parameter for the termination condition
# h is the distance for the finite differences step
def gradient_ascent_with_gradient( f, fdot, t, x0, eps = 1e-3, h = 1., t_div = 25., MAX_ITER = 100, verbose = True ):
    
    if verbose:
        tic = time.time()
        
    # TODO
            
    if verbose:
        toc = time.time()
        
        print('total running time is {} seconds.\n'.format(toc - tic))
    
    return xopt

In [ ]:
clf = svm.SVC(C = 2.0, kernel = 'linear')
clf.fit(X,y)
    
loss, hinge = calculate_hinge_loss(clf,X2,y2)
print('original hinge loss is {}\n\n'.format(loss))

# initialize xc and yc
Xn = X[y < 0]
xc = Xn[0]
yc = 1

L = lambda xc : loss_function_SVM(X,y,xc,yc,X2,y2,C = 2.0)
gradL = lambda xc : loss_gradient_approximation(X,y,xc,yc,X2,y2,C = 2.0)
t = 5.
xopt = gradient_ascent_with_gradient(L,gradL,t,xc)
print('gradient ascent (using Biggio et al approximation) returns point \n{}\nwith loss {}'.format(xopt,L(xopt)))

# Task: optimize tuning parameters

Once again, play with the parameters and see how you can improve your results! Additionally, if you want to run gradient ascent from multiple initializations, feel free to do that as well.

In [ ]:
# play with parameters
# TODO

# xopt = gradient_ascent_with_gradient(L,gradL,t,xc)
# print('gradient ascent (using Biggio et al approximation) returns point \n{}\nwith loss {}'.format(xopt,L(xopt)))

# Task: analyze results

Discuss the results of your coding. What was difficult to implement? 

Did the Biggio et al approximation make your adversary's performance better or worse? Did the computation time run faster? Were the assumptions of Biggio et al valid? 

What could improve the adversary's performance? What could decrease the adversary's computational time? How do you expect performance to vary with the dimension of the input feature?

# WRITTEN RESPONSE:

???

# Importing real data

This is the last section of the lab. Deep breaths. Your journey is nearing its end.

Now, let's try and use this method on real data! We're going to be using the MNIST (Modified National Institute of Standards and Technology) dataset of handwritten digits.

The training data can be found at https://www.kaggle.com/c/digit-recognizer/data

# Task: write a function to load in the data into `X` and `y` variables

Load in real data by implementing the function below. (See `sample_code` provided on the course website if you are uncertain.)

Note that this data takes some time to load.

In [ ]:
def load_data(filename):
    
    # TODO
    
    return X,y

In [ ]:
# load data

X,y=load_data('./data/train.csv')

# Task: Isolate only 2 digits for binary classification; separate training and validation.

All our derivations above were for the case where there were only 2 classes. As such, let's look exclusively at the digits `7` and `9`. Write code to pull out these data points. You'll find that there are a little over 4000 data points for each digit. Let's use the first 3000 as training and the last 1000+ as validation. Write code to parse these into separate variables as well.

In [ ]:
# isolate only 2 types of digits

# TODO

# To use the code below, you must define the following variables:
#   (Xtrain,ytrain)
#   (Xvalid,yvalid)

## Sample code for rendering MNIST images

Below I've included some code snippets for rendering images.

In [ ]:
# sample code for showing MNIST images

ximg = np.reshape(X1[0],(28,28))

plt.imshow(ximg, cmap=plt.get_cmap('gray'))

plt.show()

ximg = np.reshape(X2[0],(28,28))

plt.imshow(ximg, cmap=plt.get_cmap('gray'))

plt.show()

# Task: run your code on this data

If all goes well, fingers crossed, it should be relatively straightforward to call your previous functions on these new variables.

Some tips I have for you before you do so: 

1) Do not use the full dataset until the very end. The learning process takes a long time. Take about 100-200 points first and see how it goes.

2) If you get an error, try restarting the kernel and running all. Sometimes if you run a lot of cells in different orders, variables may unintentionally get overwritten. This makes sure that all the code in the cells execute in order.

3) Frequently there are functions that use variable names as input arguments. Thus, if your function only takes in `(margin_support_vectors)` as an input, it may still allow you to use the variable `(margin_indices)`, despite it not being passed in as an argument. (Since it's a variable defined globally.) Make sure to only use variables passed in as an argument.

In [ ]:
tic = time.time()

clf = svm.SVC(C = 2.0, kernel = 'linear')
clf.fit(Xtrain,ytrain)
    
loss, hinge = calculate_hinge_loss(clf,Xvalid,yvalid)

toc = time.time()

print('original hinge loss is {}\n'.format(loss))
print('Computation time is {} seconds.'.format(toc - tic))

In [ ]:
# initialize xc and yc
xc = Xtrain[0]
yc = 1

tic = time.time()

L = lambda xc : loss_function_SVM(Xtrain,ytrain,xc,yc,Xvalid,yvalid,C = 2.0)
gradL = lambda xc : loss_gradient_approximation(Xtrain,ytrain,xc,yc,Xvalid,yvalid,C = 2.0)
t = 5.

xopt = gradient_ascent_with_gradient(L,gradL,t,xc, verbose = False)

toc = time.time()
print('gradient ascent (using Biggio et al approximation) returns with loss {}'.format(L(xopt)))
print('Computation time is {} seconds.'.format(toc - tic))

# Task: plot the adversarial examples

Plot the adversarial example your algorithm has come up with! Also, plot the **difference** between the initial data and the adversarial example. Whereas the examples may look relatively similar to the naked eye, the difference points out interesting 'directions' in which the SVM is more tricked.

In [ ]:
# Plot results

# TODO

# Task: verify performance on test data

As mentioned in lecture, the test data is something that should never be used until the very final moment.

As a true evaluation of how well our adversarial data poisoning algorithm works in the 'wild', let's see what the hinge loss is on the test data, which has never before been by our algorithm.

Load the test data and evaluate the hinge loss of the unpoisoned classifier, as well as the hinge loss of the poisoned classifier.

In [ ]:
# load the test data and look at the difference in hinge loss between your poisoned and unpoisoned classifier

# TODO

# Closing remarks

In this lab, we've learned how to train SVMs, how to use finite differences to do approximate gradient ascent, how to use a technique from a recent research paper to approximate the gradient when the dimensions are too large, and looked at what adversarial examples look like on the MNIST data. Additionally, we looked at how well the adversarial examples poisoned our classifier by comparing it to the sacred test data, which was not loaded at all until the very end.

Hopefully, you've also got some hands-on experience with how much of machine learning actually consists of waiting for the machine to learn.

Here's a chance for you to write down your thoughts about this lab. Summarize your process throughout this lab, and write down any things you realized implementing this lab? Additionally, do you have any feedback about the lab?

# WRITTEN RESPONSE:

???